In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math

In [ ]:
template_path = Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/\
'beamform_signals'/'220311_dl_grid_87deg_100eV_3cm_range.npy'

signal_path = Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/\
'beamform_signals'/'220315_dl_grid_87deg_100eV_3cm_range_test.npy'

In [ ]:
templates = np.load(template_path)
signals = np.load(signal_path)

In [ ]:
var = 10 * 50 * 60 * 200e6 * 1.38e-23
templates = (
    1 / np.sqrt(var * (abs(templates)**2).sum(-1))).reshape((templates.shape[0], 1)
                                                           ) * templates

In [ ]:
print(templates.shape, signals.shape)

In [ ]:
signal_scores = np.matmul(signals, templates.conjugate().T)

In [ ]:
signal_scores = abs(signal_scores)

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('plasma', as_cmap=True)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(
    signal_scores, 
    interpolation='none',
    aspect='auto',
    cmap=cmap
)
cbar = fig.colorbar(img)

In [ ]:
ideal_templates = (
    1 / np.sqrt(var * (abs(signals)**2).sum(-1))).reshape((signals.shape[0], 1)
                                                           ) * signals
ideal_scores = abs(np.matmul(signals, ideal_templates.conjugate().T))

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('plasma', as_cmap=True)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(
    ideal_scores, 
    interpolation='none',
    aspect='auto',
    cmap=cmap
)
cbar = fig.colorbar(img)

In [ ]:
ideal_scores_diag = np.diag(ideal_scores)

In [ ]:
best_scores = np.max(signal_scores, axis=-1)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


#hist_out1 = ax.hist(output[target_1, 1], 50, density=True)
hist_out0 = ax.hist(ideal_scores_diag, 50, density=True)
hist_out1 = ax.hist(best_scores, 50, density=True)
#hist_tar = ax.hist(, 50, density=True)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(ideal_scores_diag, best_scores, '.')

In [ ]:
x = np.linspace(0, 12, 201)

noise_pdf = stats.rayleigh.pdf(x, loc=0, scale=(1/math.sqrt(2)))


match = 0.25

signal_pdf = stats.rice.pdf(
    x.reshape((1, x.size)),
    match * best_scores.flatten().reshape(best_scores.size, 1),
    loc = match * best_scores.flatten().reshape(best_scores.size, 1) * (1 - 1 / math.sqrt(2)),
    scale=1/math.sqrt(2)
)

In [ ]:
x = np.linspace(0, 12, 201)

noise_sf = stats.rayleigh.sf(x, loc=0, scale=(1/math.sqrt(2)))


match = 0.25

signal_sf = stats.rice.sf(
    x.reshape((1, x.size)),
    match * best_scores.flatten().reshape(best_scores.size, 1),
    loc = match * best_scores.flatten().reshape(best_scores.size, 1) * (1 - 1 / math.sqrt(2)),
    scale=1/math.sqrt(2)
)

In [ ]:
print(signal_sf.shape, signal_pdf.shape)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(x, signal_pdf.mean(axis=0))
ax.plot(x, noise_pdf)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(noise_sf, signal_sf.mean(axis=0))
y = np.linspace(0, 1, 10000)
ax.plot(y, y)
ax.set_xscale('log')
ax.set_xlim(1e-3, 1)

# find mean match for different numbers of templates

In [ ]:

match = []
number_templates = []

for n in range(signal_scores.shape[-1]//2):

    selection_inds = np.arange(0, signal_scores.shape[-1], n+1)
    
    select_scores = signal_scores[:, selection_inds]
    
    select_best_scores = np.max(select_scores, axis=-1)
    
    match.append(np.mean(select_best_scores/ideal_scores_diag))
    number_templates.append(selection_inds.size)
    
    if n % 2000 == 1999:
        print(n+1, ' ', 1+signal_scores.shape[-1]//2)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(number_templates, match, '.')
#ax.set_xlim(100, 1000)
ax.set_xscale('log')

# compare

In [ ]:
dnn_result_path = Path.home()/'group'/'project'/'results'/'machine_learning'/\
'dnn'/'triggering'/'distributions'/'220317_dl_grid_87_100_cnn_v1_test_output_dist.npz'
dnn_dist = np.load(dnn_result_path)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(noise_sf, signal_sf.mean(axis=0))
ax.plot(1-dnn_dist['noise_cdf'], 1-dnn_dist['signal_cdf'])
y = np.linspace(0, 1, 10000)
ax.plot(y, y)
ax.set_xscale('log')
ax.set_xlim(1e-3, 1)